In [23]:
import os

In [24]:
os.getcwd()

'd:\\ML Projects\\house_price_prediction'

In [3]:
os.chdir("d:\\ML Projects\\house_price_prediction")

In [4]:
os.getcwd()

'd:\\ML Projects\\house_price_prediction'

In [5]:
from housing.config.configuration import Configuration

In [8]:
config = Configuration()

In [9]:
data_ingestion_config = config.get_data_ingestion_config()

In [10]:
data_validation_config = config.get_data_validation_config()

In [11]:
data_transformation_config = config.get_data_transformation_config()

In [12]:
from housing.component.data_ingestion import DataIngestion

In [13]:
data_ingestion = DataIngestion(data_ingestion_config)

In [14]:
data_ingestion_artifact = data_ingestion.initiate_data_ingestion()

In [15]:
data_ingestion_artifact

DataIngestionArtifact(train_file_path='d:\\ML Projects\\house_price_prediction\\housing\\artifact\\data_ingestion\\2022-07-13_19-40-58\\ingested_data\\train\\housing.csv', test_file_path='d:\\ML Projects\\house_price_prediction\\housing\\artifact\\data_ingestion\\2022-07-13_19-40-58\\ingested_data\\test\\housing.csv', is_ingested=True, message='Data Ingestion completed successfully')

In [16]:
from housing.component.data_validation import DataValidation

In [17]:
data_validation = DataValidation(data_validation_config,data_ingestion_artifact)

In [18]:
data_validation_artifact = data_validation.initiate_data_validation()

Total Number Of Columns = [10]
            Columns Names are : [Index(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income',
       'median_house_value', 'ocean_proximity'],
      dtype='object')],
            Numerical Feature Columns :[['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income', 'median_house_value']],
            Categorical Features Column : [['ocean_proximity']],
            Values in Ocean Proximity Column : [<1H OCEAN     7277
INLAND        5262
NEAR OCEAN    2124
NEAR BAY      1847
ISLAND           2
Name: ocean_proximity, dtype: int64]
            Validation Status =[True]
            


In [25]:
data_validation_artifact

DataValidationArtifact(schema_file_path='d:\\ML Projects\\house_price_prediction\\config\\schema.yaml', report_file_path='d:\\ML Projects\\house_price_prediction\\housing\\artifact\\data-validation\\2022-07-13_19-40-58\\report.json', report_page_file_path='d:\\ML Projects\\house_price_prediction\\housing\\artifact\\data-validation\\2022-07-13_19-40-58\\report.html', is_validated=True, message='Data validation Performed Successfuly')

In [20]:
from housing.component.data_transformation import DataTransformation

In [21]:
data_transformation = DataTransformation(config,data_transformation_config,data_ingestion_artifact,data_validation_artifact)

In [26]:
data_transformer_object = data_transformation.get_data_transformer_object()

In [27]:
data_transformer_object

ColumnTransformer(transformers=[('num_pipeline',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('feature_generator',
                                                  FeatureGenerator(columns=['longitude',
                                                                            'latitude',
                                                                            'housing_median_age',
                                                                            'total_rooms',
                                                                            'total_bedrooms',
                                                                            'population',
                                                                            'households',
                                                                            'median_income',
                                                                            'median_house_value'])),
                                                 ('scaler', StandardScaler())]),
                                 ['longitude', 'latitude', 'housing_median_age',
                                  'total_rooms', 'total_bedrooms', 'population',
                                  'households', 'median_income',
                                  'median_house_value']),
                                ('cat_pipeline',
                                 Pipeline(steps=[('impute',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('one_hot_encoder',
                                                  OneHotEncoder()),
                                                 ('scaler',
                                                  StandardScaler(with_mean=False))]),
                                 'ocean_proximity')])

In [28]:
import logging
from housing.exception import ExceptionHendler

In [ ]:
from housing.entity.artifact_entity import DataTransformationArtifact
from housing.util.util import read_yaml_file
import numpy as np

In [ ]:
def initiate_data_transformation(self)->DataTransformationArtifact:
        try:
            logging.info(f"Obtaining preprocessing object.")
            preprocessing_obj = self.get_data_transformer_object()


            logging.info(f"Obtaining training and test file path.")
            train_file_path = self.data_ingestion_artifact.train_file_path
            test_file_path = self.data_ingestion_artifact.test_file_path
            

            schema_file_path = self.data_validation_artifact.schema_file_path
            
            logging.info(f"Loading training and test data as pandas dataframe.")
            train_df = load_data(file_path=train_file_path, schema_file_path=schema_file_path)
            
            test_df = load_data(file_path=test_file_path, schema_file_path=schema_file_path)

            schema = read_yaml_file(file_path=schema_file_path)

            target_column_name = schema[TARGET_COLUMN_KEY]


            logging.info(f"Splitting input and target feature from training and testing dataframe.")
            input_feature_train_df = train_df.drop(columns=[target_column_name],axis=1)
            target_feature_train_df = train_df[target_column_name]

            input_feature_test_df = test_df.drop(columns=[target_column_name],axis=1)
            target_feature_test_df = test_df[target_column_name]
            

            logging.info(f"Applying preprocessing object on training dataframe and testing dataframe")
            input_feature_train_arr=preprocessing_obj.fit_transform(input_feature_train_df)
            input_feature_test_arr = preprocessing_obj.transform(input_feature_test_df)


            train_arr = np.c_[ input_feature_train_arr, np.array(target_feature_train_df)]

            test_arr = np.c_[input_feature_test_arr, np.array(target_feature_test_df)]
            
            transformed_train_dir = self.data_transformation_config.transformed_train_dir
            transformed_test_dir = self.data_transformation_config.transform_test_dir

            train_file_name = os.path.basename(train_file_path).replace(".csv",".npz")
            test_file_name = os.path.basename(test_file_path).replace(".csv",".npz")

            transformed_train_file_path = os.path.join(transformed_train_dir, train_file_name)
            transformed_test_file_path = os.path.join(transformed_test_dir, test_file_name)

            logging.info(f"Saving transformed training and testing array.")
            
            save_numpy_array_data(file_path=transformed_train_file_path,array=train_arr)
            save_numpy_array_data(file_path=transformed_test_file_path,array=test_arr)

            preprocessing_obj_file_path = self.data_transformation_config.preprocessed_object_fle_path

            logging.info(f"Saving preprocessing object.")
            save_object(file_path=preprocessing_obj_file_path,obj=preprocessing_obj)

            data_transformation_artifact = DataTransformationArtifact(is_transformed=True,
            message="Data transformation successfull.",
            transformed_train_file_path=transformed_train_file_path,
            transformed_test_file_path=transformed_test_file_path,
            preprocessed_object_file_path=preprocessing_obj_file_path

            )
            logging.info(f"Data transformationa artifact: {data_transformation_artifact}")
            return data_transformation_artifact
        except Exception as e:
            raise HousingException(e,sys) from e
